## *RISK* attrition

[StackExchange question.](https://math.stackexchange.com/questions/4318008/comparing-the-probability-of-parallel-multiple-dice-rolls)

* Two sides each have a pool of d6s.
* Each side rolls their dice, and then pairs of one die from each side are made from highest to lowest.
    (Some dice may be unpaired.)
* For each pair, if one side rolled a higher number, they eliminate the opposing die.
    On a tie nothing happens to that pair.
* Repeat until one side runs out of dice.

In [1]:
import piplite
await piplite.install("icepool")

import icepool
from icepool import d6

class EvalRiskAttrition(icepool.EvalPool):
    def next_state(self, state, outcome, a, b):
        if state is None:
            score_a, score_b, advantage = 0, 0, 0
        else:
            score_a, score_b, advantage = state
        # Advantage is the number of unpaired dice that rolled a previous (higher) number.
        # If positive, it favors side A, otherwise it favors side B.
        # We pair them off with newly-rolled dice of the disadvantaged side.
        if advantage > 0:
            score_a += min(b, advantage)
        elif advantage < 0:
            score_b += min(a, -advantage)
        advantage += a - b
        return score_a, score_b, advantage
    
    def final_outcome(self, final_state, pool_a, pool_b):
        score_a, score_b, advantage = final_state
        if score_a == 0 and score_b == 0:
            # No change. Eliminate this outcome to prevent infinite looping.
            # This is equivalent to rerolling the contest until at least one die is removed.
            return icepool.Reroll
        # Each side loses dice equal to the other's hits.
        # The result is the number of remaining dice on each side.
        return pool_a.num_dice() - score_b, pool_b.num_dice() - score_a
    
    def direction(self, *_):
        # See outcomes in descending order.
        return -1

eval_risk = EvalRiskAttrition()

def risk_attrition(state):
    a, b = state
    if a == 0 or b == 0:
        # If one side has run out of dice, no more rolling is necessary.
        return a, b
    else:
        # Otherwise, run the contest.
        return eval_risk(d6.pool(a), d6.pool(b))

# 4 dice vs. 3 dice.
a = 4
b = 3
# Construct a die that always rolls the tuple (4, 3).
# Then, apply the risk_attrition function recursively until reaching a fixed point.
result = icepool.Die((a, b)).sub(risk_attrition, max_depth=None, denominator_method='reduce')
# The result is how many dice are remaining at the end.
# The loser has 0 dice, and the winner has 1 or more dice.
print(result)

<class 'AttributeError'>: 'collections.defaultdict' object has no attribute 'reduce'

In [2]:
# If we just want to know the winner:
print(result.sub(lambda a, b: 'a' if a > 0 else 'b').reduce())

<class 'NameError'>: name 'result' is not defined

### Versus Monte Carlo

We can compare this to a Monte Carlo simulation, which is slower and noisier, but provides an independent alternative.

In [3]:
from collections import Counter

def monte_carlo(a, b):
    if a == 0 or b == 0:
        return a, b
    else:
        rolls_a = sorted((icepool.d6.sample() for i in range(a)), reverse=True)
        rolls_b = sorted((icepool.d6.sample() for i in range(b)), reverse=True)
        for roll_a, roll_b in zip(rolls_a, rolls_b):
            if roll_a > roll_b: b -= 1
            if roll_b > roll_a: a -= 1
        return monte_carlo(a, b)

counts = Counter()

for i in range(10000):
    counts[monte_carlo(a, b)] += 1

for key in sorted(counts.keys()):
    print(key, counts[key])


(0, 1) 287
(0, 2) 763
(0, 3) 1261
(1, 0) 278
(2, 0) 774
(3, 0) 1725
(4, 0) 4912
